### Farm Planning ###

In [2]:

from pyomo.environ import *

# Modelo
model = ConcreteModel()

# Conjuntos
years = range(1, 6)
lands = range(1, 5)
ages = range(1, 13)
cow_ages = range(2, 12)

model.Years = Set(initialize=years)
model.Lands = Set(initialize=lands)
model.Ages = Set(initialize=ages)
model.CowAges = Set(initialize=cow_ages)

# Parámetros
gr_area = {1: 20.0, 2: 30.0, 3: 20.0, 4: 10.0}
gr_yield = {1: 1.1, 2: 0.9, 3: 0.8, 4: 0.65}
sb_yield = 1.5
housing_cap = 130
gr_intake = 0.6
sb_intake = 0.7
hf_land = 2 / 3.0
land_cap = 200
hf_labor = 0.1
cow_labor = 0.42
gr_labor = 0.04
sb_labor = 0.14
labor_cap = 55.0
cow_decay = 0.02
hf_decay = 0.05
initial_hf = 5.0       # reducido para factibilidad
initial_cows = 5.0     # reducido para factibilidad
birthrate = 1.1
min_final_cows = 10    # relajado
max_final_cows = 150
bl_price = 30
hf_price = 40
cow_price = 120
milk_price = 370
gr_price = 75
sb_price = 58
gr_cost = 90
sb_cost = 70
overtime_cost = 120
regular_time_cost = 4000
hf_cost = 50
cow_cost = 100
gr_land_cost = 15
sb_land_cost = 10
installment = 39.71

# Variables
model.sb = Var(years, domain=NonNegativeReals)
model.gr_buy = Var(years, domain=NonNegativeReals)
model.gr_sell = Var(years, domain=NonNegativeReals)
model.sb_buy = Var(years, domain=NonNegativeReals)
model.sb_sell = Var(years, domain=NonNegativeReals)
model.overtime = Var(years, domain=NonNegativeReals)
model.outlay = Var(years, domain=NonNegativeReals)
model.hf_sell = Var(years, domain=NonNegativeReals)
model.newborn = Var(years, domain=NonNegativeReals)
model.profit = Var(years, domain=Reals)
model.gr = Var(years, lands, domain=NonNegativeReals)
model.cows = Var(years, ages, domain=NonNegativeReals)

# Restricciones
def housing_rule(m, t):
    return m.newborn[t] + m.cows[t,1] + sum(m.cows[t,k] for k in cow_ages) <= housing_cap + sum(m.outlay[y] for y in years if y <= t)
model.Housing = Constraint(years, rule=housing_rule)

def grain_food(m, t):
    return sum(gr_intake * m.cows[t,k] for k in cow_ages) <= m.gr_buy[t] - m.gr_sell[t] + sum(m.gr[t,l] for l in lands)
model.GrainFood = Constraint(years, rule=grain_food)

def sb_food(m, t):
    return sum(sb_intake * m.cows[t,k] for k in cow_ages) <= m.sb_buy[t] - m.sb_sell[t] + m.sb[t]
model.SBFood = Constraint(years, rule=sb_food)

def grain_capacity(m, t, l):
    return m.gr[t,l] <= gr_yield[l] * gr_area[l]
model.GrainCap = Constraint(years, lands, rule=grain_capacity)

def land_rule(m, t):
    return m.sb[t]/sb_yield + hf_land*(m.newborn[t] + m.cows[t,1]) + sum(m.gr[t,l]/gr_yield[l] for l in lands) + sum(m.cows[t,k] for k in cow_ages) <= land_cap
model.Land = Constraint(years, rule=land_rule)

def labor_rule(m, t):
    return hf_labor*(m.newborn[t] + m.cows[t,1]) + sum(cow_labor * m.cows[t,k] for k in cow_ages) + sum(gr_labor/gr_yield[l]*m.gr[t,l] for l in lands) + sb_labor/sb_yield*m.sb[t] <= labor_cap + m.overtime[t]
model.Labor = Constraint(years, rule=labor_rule)

def continuity1(m, t):
    if t == 1:
        return Constraint.Skip
    return m.cows[t,1] == (1 - hf_decay) * m.newborn[t-1]
model.Cont1 = Constraint(years, rule=continuity1)

def continuity2(m, t):
    if t == 1:
        return Constraint.Skip
    return m.cows[t,2] == (1 - hf_decay) * m.cows[t-1,1]
model.Cont2 = Constraint(years, rule=continuity2)

def continuity3(m, t, k):
    if t == 1 or k >= 11:
        return Constraint.Skip
    return m.cows[t,k+1] == (1 - cow_decay) * m.cows[t-1,k]
model.Cont3 = Constraint(years, cow_ages, rule=continuity3)

def heifer_birth(m, t):
    return m.newborn[t] + m.hf_sell[t] == sum(birthrate/2 * m.cows[t,k] for k in cow_ages)
model.HeiferBirth = Constraint(years, rule=heifer_birth)

def final_cows(m):
    return inequality(min_final_cows, sum(m.cows[5,k] for k in cow_ages), max_final_cows)
model.FinalCows = Constraint(rule=final_cows)

def init_heifers(m, k):
    if k < 3:
        return m.cows[1,k] == initial_hf
    return Constraint.Skip
model.InitH = Constraint(model.Ages, rule=init_heifers)

def init_cows(m, k):
    if k >= 3:
        return m.cows[1,k] == initial_cows
    return Constraint.Skip
model.InitC = Constraint(model.Ages, rule=init_cows)

def yearly_profit(m, t):
    return m.profit[t] == (
        bl_price * birthrate/2 * sum(m.cows[t,k] for k in cow_ages)
        + hf_price * m.hf_sell[t]
        + cow_price * m.cows[t,12]
        + milk_price * sum(m.cows[t,k] for k in cow_ages)
        + gr_price * m.gr_sell[t]
        + sb_price * m.sb_sell[t]
        - gr_cost * m.gr_buy[t]
        - sb_cost * m.sb_buy[t]
        - overtime_cost * m.overtime[t]
        - regular_time_cost
        - hf_cost * (m.newborn[t] + m.cows[t,1])
        - cow_cost * sum(m.cows[t,k] for k in cow_ages)
        - gr_land_cost * sum(m.gr[t,l]/gr_yield[l] for l in lands)
        - sb_land_cost * m.sb[t]/sb_yield
        - installment * sum(m.outlay[y] for y in years if y <= t)
    )
model.YearlyProfit = Constraint(years, rule=yearly_profit)

# Objetivo
model.Objective = Objective(expr=sum(model.profit[t] - installment*(t+4)*model.outlay[t] for t in years), sense=maximize)

# Resolver
solver = SolverFactory('glpk')
results = solver.solve(model, tee=True)

# Mostrar resultados
for t in years:
    print(f"Año {t}: Ganancia = {model.profit[t]():.2f}, Gasto en alojamiento = {model.outlay[t]():.2f}")


GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write C:\Users\AMINAD~1\AppData\Local\Temp\tmpn4hms_w1.glpk.raw --wglp
 C:\Users\AMINAD~1\AppData\Local\Temp\tmp6x2j0s2b.glpk.glp --cpxlp C:\Users\AMINAD~1\AppData\Local\Temp\tmpjx3uyyzv.pyomo.lp
Reading problem data from 'C:\Users\AMINAD~1\AppData\Local\Temp\tmpjx3uyyzv.pyomo.lp'...
113 rows, 130 columns, 735 non-zeros
1223 lines were read
Writing problem data to 'C:\Users\AMINAD~1\AppData\Local\Temp\tmp6x2j0s2b.glpk.glp'...
1060 lines were written
GLPK Simplex Optimizer, v4.65
113 rows, 130 columns, 735 non-zeros
Preprocessing...
PROBLEM HAS NO DUAL FEASIBLE SOLUTION
If you need actual output for non-optimal solution, use --nopresol
Time used:   0.0 secs
Memory used: 0.2 Mb (174612 bytes)
Writing basic solution to 'C:\Users\AMINAD~1\AppData\Local\Temp\tmpn4hms_w1.glpk.raw'...
252 lines were written


TypeError: unsupported format string passed to NoneType.__format__

In [3]:
from pyomo.environ import *

# Crear modelo
model = ConcreteModel()

# Conjuntos
model.Years = RangeSet(1, 5)
model.Lands = RangeSet(1, 4)
model.Ages = RangeSet(1, 12)
cow_ages = list(range(2, 12))

# Parámetros
model.housing_cap = Param(initialize=130)
model.land_cap = Param(initialize=200)
model.labor_cap = Param(initialize=5500)
model.installment = Param(initialize=39.71)

model.grain_intake = Param(initialize=0.6)
model.sugar_intake = Param(initialize=0.7)

model.heifer_decay = Param(initialize=0.05)
model.cow_decay = Param(initialize=0.02)
model.birthrate = Param(initialize=1.1)

model.milk_price = Param(initialize=370)
model.bl_price = Param(initialize=30)
model.hf_price = Param(initialize=40)
model.cow_price = Param(initialize=120)

model.gr_cost = Param(initialize=90)
model.sb_cost = Param(initialize=70)
model.gr_price = Param(initialize=75)
model.sb_price = Param(initialize=58)

model.overtime_cost = Param(initialize=120)
model.regular_time_cost = Param(initialize=4000)

model.hf_cost = Param(initialize=50)
model.cow_cost = Param(initialize=100)
model.gr_land_cost = Param(initialize=15)
model.sb_land_cost = Param(initialize=10)

model.sb_yield = Param(initialize=1.5)
grain_yield = {1: 1.10, 2: 0.90, 3: 0.80, 4: 0.65}
grain_area = {1: 20, 2: 30, 3: 20, 4: 10}
model.gr_yield = Param(model.Lands, initialize=grain_yield)
model.gr_area = Param(model.Lands, initialize=grain_area)

model.hf_land = Param(initialize=2/3)
model.hf_labor = Param(initialize=10)
model.cow_labor = Param(initialize=42)
model.gr_labor = Param(initialize=4)
model.sb_labor = Param(initialize=14)

model.min_final_cows = Param(initialize=60)
model.max_final_cows = Param(initialize=175)

model.initial_hf = Param(initialize=9.5)
model.initial_cows = Param(initialize=9.8)

# Variables de decisión
model.Outlay = Var(model.Years, domain=NonNegativeReals)
model.Overtime = Var(model.Years, domain=NonNegativeReals)
model.Newborn = Var(model.Years, domain=NonNegativeReals)
model.HeiferSell = Var(model.Years, domain=NonNegativeReals)
model.Profit = Var(model.Years, domain=Reals)
model.Cows = Var(model.Years, model.Ages, domain=NonNegativeReals)
model.GR = Var(model.Years, model.Lands, domain=NonNegativeReals)
model.SB = Var(model.Years, domain=NonNegativeReals)
model.GR_buy = Var(model.Years, domain=NonNegativeReals)
model.GR_sell = Var(model.Years, domain=NonNegativeReals)
model.SB_buy = Var(model.Years, domain=NonNegativeReals)
model.SB_sell = Var(model.Years, domain=NonNegativeReals)

# Función objetivo
def objective_rule(m):
    return sum(m.Profit[t] - m.installment * (t + 4) * m.Outlay[t] for t in m.Years)
model.Objective = Objective(rule=objective_rule, sense=maximize)

# Restricciones

# Capacidad de alojamiento
def housing_capacity_rule(m, t):
    return m.Newborn[t] + sum(m.Cows[t, k] for k in m.Ages if k != 12) <=            m.housing_cap + sum(m.Outlay[d] for d in m.Years if d <= t)
model.HousingConstraint = Constraint(model.Years, rule=housing_capacity_rule)

# Alimentación - Grano
def grain_rule(m, t):
    return sum(m.grain_intake * m.Cows[t, k] for k in cow_ages) <=            m.GR_buy[t] - m.GR_sell[t] + sum(m.GR[t, l] for l in m.Lands)
model.GrainConstraint = Constraint(model.Years, rule=grain_rule)

# Alimentación - Remolacha
def sugar_rule(m, t):
    return sum(m.sugar_intake * m.Cows[t, k] for k in cow_ages) <=            m.SB_buy[t] - m.SB_sell[t] + m.SB[t]
model.SugarConstraint = Constraint(model.Years, rule=sugar_rule)

# Capacidad de grano por grupo de tierra
def grain_capacity_rule(m, t, l):
    return m.GR[t, l] <= m.gr_yield[l] * m.gr_area[l]
model.GrainCapacityConstraint = Constraint(model.Years, model.Lands, rule=grain_capacity_rule)

# Capacidad de terreno total
def land_use_rule(m, t):
    return m.SB[t] / m.sb_yield + m.hf_land * (m.Newborn[t] + m.Cows[t, 1]) +            sum(m.GR[t, l] / m.gr_yield[l] for l in m.Lands) +            sum(m.Cows[t, k] for k in cow_ages) <= m.land_cap
model.LandConstraint = Constraint(model.Years, rule=land_use_rule)

# Restricción de mano de obra
def labor_rule(m, t):
    return m.hf_labor * (m.Newborn[t] + m.Cows[t, 1]) +            sum(m.cow_labor * m.Cows[t, k] for k in cow_ages) +            sum(m.gr_labor / m.gr_yield[l] * m.GR[t, l] for l in m.Lands) +            m.sb_labor / m.sb_yield * m.SB[t] <= m.labor_cap + m.Overtime[t]
model.LaborConstraint = Constraint(model.Years, rule=labor_rule)

# Continuidad del hato
def continuity_rule1(m, t):
    if t == 1:
        return Constraint.Skip
    return m.Cows[t, 1] == (1 - m.heifer_decay) * m.Newborn[t - 1]
model.Continuity1 = Constraint(model.Years, rule=continuity_rule1)

def continuity_rule2(m, t):
    if t == 1:
        return Constraint.Skip
    return m.Cows[t, 2] == (1 - m.heifer_decay) * m.Cows[t - 1, 1]
model.Continuity2 = Constraint(model.Years, rule=continuity_rule2)

def continuity_rule3(m, t, k):
    if t == 1 or k >= 11:
        return Constraint.Skip
    return m.Cows[t, k + 1] == (1 - m.cow_decay) * m.Cows[t - 1, k]
model.Continuity3 = Constraint(model.Years, cow_ages, rule=continuity_rule3)

# Nacimiento de vaquillas
def birth_rule(m, t):
    return m.Newborn[t] + m.HeiferSell[t] == sum(m.birthrate / 2 * m.Cows[t, k] for k in cow_ages)
model.BirthConstraint = Constraint(model.Years, rule=birth_rule)

# Condición final
def final_cows_rule(m):
    return inequality(m.min_final_cows, sum(m.Cows[5, k] for k in cow_ages), m.max_final_cows)
model.FinalCowsConstraint = Constraint(rule=final_cows_rule)

# Condiciones iniciales
def init_heifers(m, k):
    if k < 3:
        return m.Cows[1, k] == m.initial_hf
    return Constraint.Skip
model.InitHeifers = Constraint(model.Ages, rule=init_heifers)

def init_cows(m, k):
    if k >= 3:
        return m.Cows[1, k] == m.initial_cows
    return Constraint.Skip
model.InitCows = Constraint(model.Ages, rule=init_cows)

# Ganancia anual
def profit_rule(m, t):
    return m.Profit[t] ==         m.bl_price * m.birthrate / 2 * sum(m.Cows[t, k] for k in cow_ages) +         m.hf_price * m.HeiferSell[t] +         m.cow_price * m.Cows[t, 12] +         m.milk_price * sum(m.Cows[t, k] for k in cow_ages) +         m.gr_price * m.GR_sell[t] + m.sb_price * m.SB_sell[t] -         m.gr_cost * m.GR_buy[t] - m.sb_cost * m.SB_buy[t] -         m.overtime_cost * m.Overtime[t] - m.regular_time_cost -         m.hf_cost * (m.Newborn[t] + m.Cows[t, 1]) -         m.cow_cost * sum(m.Cows[t, k] for k in cow_ages) -         m.gr_land_cost * sum(m.GR[t, l] / m.gr_yield[l] for l in m.Lands) -         m.sb_land_cost * m.SB[t] / m.sb_yield -         m.installment * sum(m.Outlay[y] for y in m.Years if y <= t)
model.ProfitConstraint = Constraint(model.Years, rule=profit_rule)

# Resolver
solver = SolverFactory('glpk')
results = solver.solve(model, tee=True)

# Mostrar resultados
model.Profit.display()

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write C:\Users\AMINAD~1\AppData\Local\Temp\tmp3i9zq67l.glpk.raw --wglp
 C:\Users\AMINAD~1\AppData\Local\Temp\tmpfimk2y6v.glpk.glp --cpxlp C:\Users\AMINAD~1\AppData\Local\Temp\tmpo1da6iab.pyomo.lp
Reading problem data from 'C:\Users\AMINAD~1\AppData\Local\Temp\tmpo1da6iab.pyomo.lp'...
113 rows, 130 columns, 735 non-zeros
1223 lines were read
Writing problem data to 'C:\Users\AMINAD~1\AppData\Local\Temp\tmpfimk2y6v.glpk.glp'...
1060 lines were written
GLPK Simplex Optimizer, v4.65
113 rows, 130 columns, 735 non-zeros
Preprocessing...
PROBLEM HAS NO DUAL FEASIBLE SOLUTION
If you need actual output for non-optimal solution, use --nopresol
Time used:   0.0 secs
Memory used: 0.2 Mb (174612 bytes)
Writing basic solution to 'C:\Users\AMINAD~1\AppData\Local\Temp\tmp3i9zq67l.glpk.raw'...
252 lines were written
Profit : Size=5, Index=Years
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :  None :  No